In [ ]:
import numpy as np
from PIL import Image

def compress_channel(channel: np.ndarray, k: int) -> np.ndarray:
    """
    Compress one 2D channel with economy SVD and k singular values.
    """
    # Compute economy SVD
    U, S, Vt = np.linalg.svd(channel, full_matrices=False)
    # Truncate to k
    Uk = U[:, :k]
    Sk = S[:k]
    Vtk = Vt[:k, :]
    # Reconstruct
    return (Uk * Sk) @ Vtk

def compress_image_svd(input_path: str, output_path: str, k: int):
    """
    Compress an image by performing economy SVD on each channel
    and keeping only the top k singular values.
    """
    # 1) Load image and convert to float
    img = Image.open(input_path)
    arr = np.asarray(img, dtype=float)

    # 2) Handle grayscale vs RGB
    if arr.ndim == 2:
        # single-channel
        comp = compress_channel(arr, k)
        comp = np.clip(comp, 0, 255)
        out = Image.fromarray(comp.astype(np.uint8))
    else:
        # multi-channel (e.g. RGB)
        h, w, c = arr.shape
        comp = np.zeros_like(arr)
        for ch in range(c):
            comp[:, :, ch] = compress_channel(arr[:, :, ch], k)
        comp = np.clip(comp, 0, 255)
        out = Image.fromarray(comp.astype(np.uint8))

    # 3) Save result
    out.save(output_path)
    print(f"Compressed image written to {output_path}")

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="Compress image via economy SVD.")
    parser.add_argument("input", help="Path to input image (PNG, JPEG, etc.)")
    parser.add_argument("output", help="Path to save compressed image")
    parser.add_argument("-k", type=int, default=50,
                        help="Number of singular values to keep")
    args = parser.parse_args()

    compress_image_svd(args.input, args.output, args.k)